Часть 1 :)))))

In [42]:
import pandas as pd
import matplotlib.pyplot
df = pd.read_csv("/Users/katakitaeva/Desktop/DPO/Grib/homeworks/HW02/S02-hw-dataset.csv")

print("head")
print(df.head())
print("info")
print(df.info())
print("describe")
print(df.describe())
print("доля пропусков")
print(df.isna().mean())
print("доля строк-дубликатов")
print(df.duplicated().mean())
if dup_count > 0:
    print("Дубликат:")
    print(df[df.duplicated(keep=False)].head())

unrealistic_age = (df['age'] > 120) | (df['age'] <= 0)
if unrealistic_age.any():
    count = unrealistic_age.sum()
    print(f"Возраст > 120 лет: {count} записей")
    print(df[unrealistic_age][['age']].head())
else:
    print("Возрастные данные корректны (нет значений > 120 лет или <= 0)")

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns # проверка числовых значений
for col in numeric_cols: # проверка на наличие отрицательных значений
    if df[col].min() < 0:
        neg_count = (df[col] < 0).sum()
        print(f"{col}: {neg_count} отрицательных значений")


head
   user_id   age country  purchases  revenue
0        1  25.0      FR          7      749
1        2  24.0      RU          5     1115
2        3  52.0      FR          7      399
3        4  31.0      RU          6      654
4        5   NaN      DE          6     1296
info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    41 non-null     int64  
 1   age        39 non-null     float64
 2   country    41 non-null     object 
 3   purchases  41 non-null     int64  
 4   revenue    41 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.7+ KB
None
describe
         user_id         age  purchases      revenue
count  41.000000   39.000000  41.000000    41.000000
mean   20.243902   36.512821   4.829268   820.048780
std    11.659289   18.304259   2.710189   613.127269
min     1.000000    5.000000  -1.000000     0.0000

В столбце age 4.9% пропусков (2 из 41 строки)
    Остальные столбцы без пропусков

2.4% дублирующих строк (1 полный дубликат из 41)
    Найдены строки 9 и 40 с одинаковыми данными
В столбце purchases 1 отрицательное значение - невозможное количество покупок